In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 4, something not right?

In [3]:
a = 0.25
num_params=4
num_sims = 100

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.93 & 0.94 & 0.93 & 0.95   \\
Model 1 & 0.05 & 0.07 & 0.06 & 0.07 & 0.05   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.05, 0.  ]), array([0.93, 0.07, 0.  ]), array([0.94, 0.06, 0.  ]), array([0.93, 0.07, 0.  ]), array([0.95, 0.05, 0.  ]), 1.8714427873189774, 5.283079467463424, 5.276201175810691)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.95 & 0.94 & 0.94 & 0.98   \\
Model 1 & 0.05 & 0.04 & 0.05 & 0.05 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.94, 0.05, 0.01]), array([0.95, 0.04, 0.01]), array([0.94, 0.05, 0.01]), array([0.94, 0.05, 0.01]), array([0.98, 0.02, 0.  ]), 2.259710440421776, 6.916139647033011, 7.091409881833634)


# a  = .25, k= 9

In [6]:
a = 0.25
num_params=9
num_sims = 100

In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.97 & 0.99 & 0.96 & 0.99   \\
Model 1 & 0.00 & 0.01 & 0.00 & 0.03 & 0.00   \\
Model 2 & 0.01 & 0.02 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.99, 0.  , 0.01]), array([0.97, 0.01, 0.02]), array([0.99, 0.  , 0.01]), array([0.96, 0.03, 0.01]), array([0.99, 0.  , 0.01]), -3.0220647032619765, 5.236344130107664, 5.66381038622029)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.03 & 0.03 & 0.03 & 0.03 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), -2.466876609807727, 6.657199768168614, 7.717131849861435)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.98 & 0.98 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), -3.338449699538242, 4.077166205928686, 4.155363682678933)


# a  = 0, k= 9

In [10]:
a = 0.00 
num_params=9
num_sims = 100

In [11]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.97 & 0.98 & 0.97 & 0.98   \\
Model 1 & 0.02 & 0.03 & 0.02 & 0.03 & 0.02   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.02, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), -3.465399936689703, 1.7782895633767204, 2.8060839335381753)


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 1.00 & 0.98 & 0.98 & 0.98   \\
Model 1 & 0.01 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.98, 0.01, 0.01]), array([1., 0., 0.]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), -3.465069246826007, 2.230909320099721, 2.8188770315392513)


# a  = .25

In [13]:
a = 0.25
num_params=19
num_sims = 100

In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.94 & 0.92 & 0.93   \\
Model 1 & 0.04 & 0.04 & 0.04 & 0.04 & 0.04   \\
Model 2 & 0.03 & 0.03 & 0.02 & 0.04 & 0.03   \\
\hline
\end{tabular}
(array([0.93, 0.04, 0.03]), array([0.93, 0.04, 0.03]), array([0.94, 0.04, 0.02]), array([0.92, 0.04, 0.04]), array([0.93, 0.04, 0.03]), -8.786849345797961, 7.1027920682289585, 6.716246382760796)


In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.99 & 0.98 & 0.98 & 0.98   \\
Model 1 & 0.02 & 0.01 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), -7.5222000299593095, 7.448524868587301, 8.406076994816766)


In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.92 & 0.93 & 0.92 & 0.93   \\
Model 1 & 0.06 & 0.07 & 0.06 & 0.07 & 0.06   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.93, 0.06, 0.01]), array([0.92, 0.07, 0.01]), array([0.93, 0.06, 0.01]), array([0.92, 0.07, 0.01]), array([0.93, 0.06, 0.01]), -5.841382533679308, 11.693416920976954, 11.145870868789473)


# a = 0

In [17]:
a = 0.0
num_params=19
num_sims = 100

In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.98 & 0.99 & 0.98   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.01 & 0.02   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.98, 0.02, 0.  ]), -8.545930986781144, 2.8530859188573943, 4.233656928178262)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.99 & 0.99 & 0.99 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), -9.105564347783021, 3.374059786254432, 4.404871281372424)


# evidence of power

In [20]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [21]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.53 & 0.49 & 0.51 & 0.49 & 0.60   \\
Model 1 & 0.47 & 0.51 & 0.49 & 0.51 & 0.40   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.53, 0.47, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.51, 0.49, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.6, 0.4, 0. ]), 6.979357403436703, 4.925982680123298, 5.215754245964105)


In [22]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.63 & 0.60 & 0.65 & 0.61 & 0.69   \\
Model 1 & 0.37 & 0.40 & 0.35 & 0.39 & 0.31   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.63, 0.37, 0.  ]), array([0.6, 0.4, 0. ]), array([0.65, 0.35, 0.  ]), array([0.61, 0.39, 0.  ]), array([0.69, 0.31, 0.  ]), 6.088128793586734, 4.910784617440597, 5.154244328999802)


In [23]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.49 & 0.50 & 0.62   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.45 & 0.45 & 0.51 & 0.50 & 0.38   \\
\hline
\end{tabular}
(array([0.55, 0.  , 0.45]), array([0.55, 0.  , 0.45]), array([0.49, 0.  , 0.51]), array([0.5, 0. , 0.5]), array([0.62, 0.  , 0.38]), -13.685559141967138, 4.8481870887803735, 5.045509028219955)


In [24]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.60 & 0.62 & 0.59 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.41 & 0.40 & 0.38 & 0.41 & 0.35   \\
\hline
\end{tabular}
(array([0.59, 0.  , 0.41]), array([0.6, 0. , 0.4]), array([0.62, 0.  , 0.38]), array([0.59, 0.  , 0.41]), array([0.65, 0.  , 0.35]), -12.855128639265535, 4.404265997010801, 5.185499295629134)
